# Connect to Postgresql
Import packages

In [1]:
import psycopg2
import psycopg2.extras
from IPython.display import Image, display, HTML

### Define Utility Functions

Define a function for displaying item images

In [2]:
def displayImageUrls(urls):
    html = ""
    for url in urls:
        html += "<img src='" + url + "' width='95' style='display:inline-block'>"
    display(HTML(html))

Assume the query returns results that include 'item_image' and add the url to images.

In [3]:
def getImageUrls(conn, query):
    images = []
    with conn:
        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
            cursor.execute(query)
            for record in cursor:
                images.append(dict(record)['item_image'])
    return images

Connecto to local db

In [4]:
conn = psycopg2.connect("dbname=fcrec user=fcuser password=fcuser123")

# Show Items
## Random Items

In [5]:
query = '''
select item_id, item_image, price, category1, category2, category3, category4, brand
from site_product 
limit 20 offset 10;
'''

displayImageUrls(getImageUrls(conn, query))

## User's Viewed Items

In [6]:
query = '''
select t1.server_time server_time, t1.session_id session_id, t1.item_id item_id, t2.item_image item_image
from view_log t1
    join product t2 on t2.item_id = t1.item_id
where uid = '00bb485'
order by server_time asc
'''

displayImageUrls(getImageUrls(conn, query))

## Most Popular Category Items

In [7]:
query = '''
select t1.item_id, t2.item_image, count(*) cnt
from view_log t1
    join product t2 on t2.item_id = t1.item_id
where t2.category3 = '301f675'
group by t1.item_id, t2.item_image
order by cnt desc
limit 20;
'''

displayImageUrls(getImageUrls(conn, query))

Close the connection

In [8]:
conn.close()